# 速率限制器

速率限制器可以控制系统的处理请求或者执行操作的频率，从而达到保护系统自身、数据及用户安全等多个方面的目的。如：
+ 通过速率限制手段限制用户在一定时间内访问页面的次数，从而避免网络爬虫行为。
+ 限制用户在一定时间内尝试登陆的次数，以此来避免黑客对用户密码进行暴力破解。
+ 云计算会用速率限制器抵御恶意的 API 调用和 DDoS 攻击。

它的逻辑就是，比如想要限制用户 Peter 在 24h 内尝试登陆的次数，那么可以在 Peter 每次尝试登陆时使用 INCR 将其对应的 `RateLimiter:Peter:login` 计数器的值加 1，并用 `NX` 的方式为键限制 24h，过了24h，自动删除该键重新计数。

使用时，通过检查计数器的当前值与系统允许的最大可执行次数来判断用户是否可执相关的操作。另外当用户由于执行过多次数被禁止时，可以针对计数器执行 `TTL` 命令来获知用户解禁所需时间，或者使用 `DEL` 命令删除计数器来解除对该用户的操作限制。

In [19]:
"""
配置连接
"""
from redis import Redis

# Redis连接配置
client = Redis(
    host='39.104.208.122', 
    port=6379,
    decode_responses=False,  # 关闭自动解码
    ssl=False
)

if client.ping():
    print("Redis连接成功")
else:
    print("Redis连接失败")

Redis连接成功


In [22]:
def make_limiter_key(uid, action):
    """
    构建用于记录用户执行指定行为次数的计数器键。
    例子： RateLimiter:Peter:login
    """
    return "RateLimiter:{0}:{1}".format(uid, action)

class RateLimiter:

    def __init__(self, client, action, interval, maximum):
        """
        根据给定的行为、间隔和最大次数参数，创建相应行为的速率限制器实例。
        """
        self.client = client
        self.action = action
        self.interval = interval
        self.maximum = maximum

    def is_permitted(self, uid):
        """
        判断给定用户当前是否可以执行指定行为。
        """
        key = make_limiter_key(uid, self.action)
        # 更新计数器并在有需要时为其设置过期时间
        tx = self.client.pipeline()
        tx.incr(key)
        # EXPIRE key interval NX
        # 本书用到的是Redis7，由于我用的是Redis6，下面的命令无法在Redis6中执行
        tx.expire(key, self.interval, nx=True)
        current_times, _ = tx.execute()
        # 根据计数器的当前值判断本次行为是否可以执行
        return current_times <= self.maximum

    def remaining(self, uid):
        """
        返回给定用户当前还可以执行指定行为的次数。
        """
        # 根据键获取计数器中储存的值
        key = make_limiter_key(uid, self.action)
        current_times = self.client.get(key)
        # 值为空则表示给定用户当前并未执行过指定行为
        if current_times is None:
            return self.maximum
        # 将值转换为数字，然后通过计算获取剩余的可执行次数
        current_times = int(current_times)
        if current_times > self.maximum:
            return 0
        else:
            return self.maximum - current_times

    def duration(self, uid):
        """
        计算距离给定用户允许再次执行指定行为需要多长时间，单位为秒。
        返回None则表示给定用户当前无需等待，仍然可以执行指定行为。
        """
        # 同时取出计数器的当前值和它的剩余生存时间
        key = make_limiter_key(uid, self.action)
        tx = self.client.pipeline()
        tx.get(key)
        # TTL key
        tx.ttl(key)
        current_times, remaining_ttl = tx.execute()
        # 仅在计数器非空并且次数已超限的情况下计算需等待时长
        if current_times is not None:
            if int(current_times) >= self.maximum:
                return remaining_ttl

    def revoke(self, uid):
        """
        撤销对用户执行指定行为的限制。
        """
        key = make_limiter_key(uid, self.action)
        self.client.delete(key)

In [ ]:
limiter = RateLimiter(client, "login", 86400, 3)

for _ in range(5):
    # 因版本差异无法执行
    print(limiter.is_permitted("Peter"))

print(limiter.duration("Peter"))